# importing:
import os library for collecting the text files.

import string and nltk for tokenizing the files.

import product for calculating cartesian product between lists.

In [ ]:
import os
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from itertools import product

# Trie:
This is the class of trie. we store every word in document and their permutation in this structure.

It has an insert function, search function.

Each node contains:
1. children dictionary: It stores every character that comes after the node's char in all documents.
2. is_end_of_word boolean: It shows that if this character is the end of some word in documents.
3. doc dictionary: every documents and their indexes that the word has appeared in them, stores in this dictionary.
4. char: it shows that which character is stored in this TrieNode.

In [ ]:
class TrieNode:
    def __init__(self, char):
        self.children = dict({})  # A dictionary to store child nodes
        self.is_end_of_word = False  # Flag to indicate if a word ends at this node
        self.doc = dict({}) # A dictionary to store the documents and the indexes that the term has appeard in that document
        self.char = char # The node of a character

class Trie:
    def __init__(self):
        self.root = TrieNode('')  # The root node of the Trie

    def insert(self, word, doc_id, index):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode(char) # If the char doesn't exist
            node = node.children[char]
        node.is_end_of_word = True
        if doc_id not in node.doc: 
            node.doc[doc_id] = [index] # If the document doesn't exists
        else:
            node.doc[doc_id].append(index)

    def search(self, word): # This function returns a last node and a bool which shows if it is a word
        node = self.root
        for char in word:
            if char not in node.children:
                return False, None
            node = node.children[char]
        return node.is_end_of_word, node

# getting files:

in this cell we use os library to get the text files in current directory.

In [ ]:
def get_text_files():
    current_directory = os.getcwd()

    # List all files in the current directory
    file_list = os.listdir(current_directory)

    # Filter the list to include only text files (if needed)
    text_files = [file for file in file_list if file.endswith('.txt')]

    # This is where we keep our documents
    documents = []

    # Loop through each text file and open/read them
    for i in range(len(text_files)):
        file_name = text_files[i]
        file_path = os.path.join(current_directory, file_name)
        
        # Open and read the file
        with open(file_path, 'r') as file:
            documents.append(file.read())
    
    return text_files, documents

# tokenize:

In this cell, every text files, convert to a list whitout any puntuation and stop word.

Punctuation are not important at all, so removing them helps the algorithm.

Removing stop words is important beacuse they don't add much value to a text and make our search more sufficient and faster.

Also we convert every character to it's lower case. And we do the same with query So that makes search and comparing more easy.

In [ ]:
def tokenize(documents):
    # Set the stop words for English
    stop_words = set(stopwords.words('english'))

    # This is the final list of tokenized text in lists
    tokenized_list = []
    
    # For each document, we use nltk regex_tokenize to token all the text file
    for doc in documents:
        tokenized_text =  nltk.regexp_tokenize(doc, r'\d+,\d+|\w+')
        
        # Here we handle ',' in the numbers (beacuse nltk doesn't handle this)
        for i in range(len(tokenized_text)):
            if ',' in tokenized_text[i]:
                w = ''
                for c in tokenized_text[i]:
                    if c != ',':
                        w += c 
                    tokenized_text[i] = w
        
        # Then remove any punctuation
        text_without_punctuation = [word.lower() for word in tokenized_text if word.isalnum()]
        
        # And remove the stop words cause they don't add much to a text
        text_without_stop_words = [word for word in text_without_punctuation if word not in stop_words]
        
        tokenized_list.append(text_without_stop_words)
    return tokenized_list

# Construct Trie:

We have two functions for constructing our Trie with every term in documents:
1. construct_trie: Using this function, we add every term and it's permutation with '$' in the Trie.
2. permute_add: This function add every permutation of a word.

In [ ]:
def permute_add(s, trie, doc_id, index):
    l = len(s)
    for i in range(l):
        trie.insert(s, doc_id, index) # Insert every permutation of s
        s = s[1:] + s[0]

In [ ]:
def construct_trie(tokenized_list):
    trie = Trie()
    
    # Constructiong the Trie
    for i in range(len(tokenized_list)):
        for j in range(len(tokenized_list[i])):
            s = tokenized_list[i][j] + "$"
            permute_add(s, trie, i, j)
    return trie

# Wildcard:

This function is used to divide a word as we like:

1. First we add a '$' to the word given
2. Then we check if the space between two '*' is more than half of the length of the word. (we use this to get the most long prefix later)
3. Then we rotate the word so a '*' will be at the end of the word. (We will have something like: word = *pref"\*"..."\*"* and we name the "\*"..."\*" part, rest. )
4. After all we divide the word to two parts (pref and rest).

In [ ]:
def clean_word(word):
    word = word + "$"
    
    # Ckeking for indexes of '*'s
    i = -1
    j = -1
    first = 1
    for k in range(len(word)):
        if (word[k] == "*" and first):
            i = k
            first = 0
        elif(word[k] == "*"):
            j = k
            break
        
    # Rotating the word as we like
    if i > 0 and j > 0:
        if (j - i > len(word)/2):
            while(word[-1] != "*"):
                word = word[1:] + word[0]
        else:
            while(word[-1] != "*"):
                word = word[-1] + word[:-1]
    
    # Rotating for one star case
    else:
        while(word[-1] != "*"):
            word = word[1:] + word[0]
    
    # finding the index of first '*' after rotation
    i = 0
    for k in range(len(word)):
        i = k
        if word[k] == "*":
            break
        
    # Dividing the word
    pref = word[:i]
    rest = word[i:]
    
    return pref, rest

This function traverse a subtrie and returns every string below the node given:

1. For each child in this node, we call the Trie_traverse with the below node and i+1. So it gives us the strings below this node. (As you can see, we get all the strings recursively.)
2. Then we check if current node is an end for some word, if it is, we add a character of the node to the result list which we will return.

Eplanation: The (i > 0) part is to add the character of the node to the returned suffixes, if it is node the given node in the first car. Cause we are trying to find every suffix for that node in the subtrie. So the concating the first node is not necessary.

In [ ]:
def Trie_traverse(node, i):
    # This is the list that we keep every suffix below this TrieNode in the Trie.
    result = []
    
    # Finding every string below and concating with the character in this node.
    for child in list(node.children.keys()):
        branch_words = Trie_traverse(node.children[child], i+1)
        result += [node.char*(i > 0) + a for a in branch_words]
    
    # If this node, is an end of a word.
    if node.is_end_of_word and i:
        result += [node.char]
        
    return result

This function is to convert the permutation to it's original (word$). And deleting the $ so we have the origial word.

In [ ]:
def make_original(term):
    # Rotating untill we have the '$' at the end of word
    while term[-1] != '$':
        term = term[-1] + term[:-1]
    
    # Returning the original word (without '$')
    return term[:-1]

This is the function that handles wildcard queries:
1. First of all we split the word as we mentioned earlier.
2. Then we search the prefix in the Trie so we will have the node of the last character of the prefix.
3. Then we use the Trie_traverse function to get every suffix in the subtrie of this node.
4. While concating the prefix to returned suffixes, if there is two '*'s in the wildcard, we check for the middle part in the suffixes. If a suffix contains the middle part (between two stars), we add it to final results.
5. And if there is nothing in the results (which means that the prefix was a word itself and had no children in the last node), we return the pref itself.

In [ ]:
def wildcard(word, trie):
    # Dividing the word
    pref, rest = clean_word(word)
    
    # Finding the prefix in trie
    is_word, node = trie.search(pref)
    
    if node:
        # Finding every suffix
        every_words = Trie_traverse(node, 0)
        
        # Concating the prefix to the suffixes
        if len(rest) > 2:
            mid = rest[1:-1]
            result = [pref + word for word in every_words if rest[1:-1] in word]
        
        else:
            result = [pref + word for word in every_words]
            
        if not len(result):
            result = [pref]
            
        return result
    
    print("there is no such a word in documents dataset!")
    return []

# Spell correction:

This sort function is to sort the spell corrections such as low distances comes first.

In [ ]:
def sort(l): # sort the spell correction list such as low distances comes first.
    for i in range(len(l)):
        for j in range(len(l)):
            if l[i][0] <  l[j][0]:
                t = l[i].copy()
                l[i] = l[j].copy()
                l[j] = t
    return l

This is the function calculates the distance between two strings using dynamic programming:

1. first construct a matrix which the charaters of the first string are on the rows and the characters of the second string are on the columns.
2. then fill the first column and row.
3. At last, fill every other element in the matrix.

(The pseudo code is available in lecture2)

In [ ]:
def edit_distance(s1, s2):
    # The function which calculates the distance between two strings.
    dp = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
    
    for i in range(1, len(s1) + 1): 
        dp[i][0] = i

    for j in range(1, len(s2) + 1): 
        dp[0][j] = j
    
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            dp[i][j] = min(dp[i-1][j] + 1, dp[i][j-1] + 1, dp[i-1][j-1] + (s1[i-1] != s2[j-1]))
    
    return dp[len(s1)][len(s2)]

This is the function that handles Spell correction:

1. First of all we find the distance between the word given and every other term in the documents.
2. While calculating the distances, we also keep the minimum distance.
3. For increased efficiency down the road, we only add the term with distance less than 5 (we have to sort this list later).
4. After sorting the best_similar list based on distance, we return terms with least distance from the word given. 

In [ ]:
def spell_correction(word, terms):
    best_similar = []
    minimum = 100
    for i in range(len(terms)):
        distance = edit_distance(word, terms[i])
        if distance < minimum:
            minimum = distance
        if (distance < 5): best_similar.append([distance, terms[i]])
    best_similar = sort(best_similar)

    return [best_similar[i][1] for i in range(len(best_similar)) if best_similar[i][0] == minimum]

# Information Retreival handling:

# Term case:
In this case, we need to find the term in the trie and then print every document in it's doc dictionary.

In [ ]:
def handling_term(term, trie):
    # Searching the term in the trie
    term += "$"
    node = trie.search(term)[1]
    
    # Printing every document in it's doc dictionary
    for doc_id in list(node.doc.keys()):
        print(text_files[doc_id][:-4])
    return

# NOT case:

In this case we have to return every other document that the term is not appeard on them.
1. First we get the documents for this term.
2. Then Print every other document that are not in the list.

In [ ]:
def handling_NOT(term, trie):
    # Searching the term and getting it's documents
    term += "$"
    docs = list(trie.search(term)[1].doc.keys())
    docs.sort()
    
    # Printing the other files than those in the docs
    i = 0
    k = 0
    key = 1
    while i < len(text_files) and k < len(docs):
        if i < docs[k]:
            print(text_files[i][:-4])
            key = 0
        
        else:
            k += 1
        i += 1
        
    while i < len(text_files):
        print(text_files[i][:-4])
        key = 0
        i += 1
    if key:
        print("no document found!")
        
    return

# And case:

In this case, we obtain the intersection between the documents of two terms.
1. Find the documents for each term.
2. Check if any document in docs1, is in docs2.
3. Print the ones that are in both lists.

In [ ]:
def handling_AND(term1, term2, trie):
    # Getting the documents for each term
    term1 += "$"
    term2 += "$"
    docs1 = list(trie.search(term1)[1].doc.keys())
    docs2 = list(trie.search(term2)[1].doc.keys())
    docs1.sort()
    docs2.sort()
    
    # Finding the intersection
    key = 1
    for i in docs1:
        if i in docs2:
            print(text_files[i][:-4])
            key = 0
    if key:
        print("no document found!")
    return

# Or case:

In this fuction, we obtain the union between documents of two terms.
1. First of all we get the documents for each term and sort them for getting union later.
2. Then print every document which any of these two terms have appeard in by getting a union between lists.

In [ ]:
def handling_OR(term1, term2, trie):
    # Finding documents
    term1 += "$"
    term2 += "$"
    docs1 = list(trie.search(term1)[1].doc.keys())
    docs2 = list(trie.search(term2)[1].doc.keys())
    docs1.sort()
    docs2.sort()
    
    # Getting union between lists
    i = 0
    j = 0
    while i < len(docs1) and j < len(docs2):
        if docs1[i] < docs2[j]:
            print(text_files[docs1[i]][:-4])
            i += 1
            
        elif docs1[i] > docs2[j]:
            print(text_files[docs2[j]][:-4])
            j += 1
            
        else:
            print(text_files[docs1[i]][:-4])
            i += 1
            j += 1
            
    while i < len(docs1):
        print(text_files[docs1[i]][:-4])
        i += 1
            
    while j < len(docs2):
        print(text_files[docs2[j]][:-4])
        j += 1
    return

# Proximity Case:

This function is used to find out if there is an index that two terms have a distance less than "distance" from each other.

In [ ]:
def distance_checking(list1, list2, distance):
    # If there is a case that two terms in a document have a distance less than "distance"
    for i in list1:
        for j in list2:
            if (abs(i-j) <= distance):
                return 1
            elif j > i:
                break
    return 0

This is a function for handling proximity:

1. First of all we find documents of two terms and sort them in ascending way.
2. Then we get the intersection between two lists.
3. While getting an intersection, we check for the proximity condition in the document.

In [ ]:
def handling_proximity(term1, term2, distance, trie):
    # Finding documents for both terms
    term1 += "$"
    term2 += "$"
    doc1 = trie.search(term1)[1].doc
    doc2 = trie.search(term2)[1].doc
    
    docs1 = list(doc1.keys())
    docs2 = list(doc2.keys())
    docs1.sort()
    docs2.sort()
    
    # Getting an intersection and checking for a proximity condition in the document
    key = 1
    for i in docs1:
        if i in docs2:
            if distance_checking(doc1[i], doc2[i], distance):
                print(text_files[i][:-4])
                key = 0
    if key:
        print("no document found!")
        
    return

# Information Retreival:

There are 3 cases in this function:

1. If nothing is entered: In this case nothing happens
2. If it is a Information retreival case: In this case we first use the wildcard or spell correction to get the correct terms and then as before, use the handling function for each correct case and return the related documents.
3. If it is just the correcting query: In this case we have to find the most correct terms based on the terms from the documents. So we use wildcard or spell correction based on the term and find the correct cases and return all of them.

In [ ]:
def information_retreival(query, trie):
    # Do nothing
    if len(query) == 0:
        return
    
    # Information retreival case
    elif len(query) == 1 or "AND" in query or "OR" in query or "NOT" in query or "NEAR" in query[1]:
        
        # Finding the correct way of each term
        result = []
        for word in query:
            if word not in ["AND", "OR", "NOT"] and "NEAR" not in word:
                if "*" in word:
                    result.append([make_original(term) for term in wildcard(word.lower(), trie)])
                else:
                    result.append(spell_correction(word.lower(), tokenized_list_stop_words))
                    
        # Handling single term
        if len(query) == 1:
            print("corrections for", query[0], ":")
            print(result[0], "\n")
            
            for word in result[0]:
                print("results for", word, ":")
                handling_term(word, trie)
            print("done!")
            return
    
        # Handling NOT case
        elif len(query) == 2:
            print("corrections for", query[1], ":")
            print(result[0], "\n")
            
            if query[0] == 'NOT':
                for word in result[0]:
                    print("results for NOT", word, ":")
                    handling_NOT(word, trie)
                print("done!")
                return
            
        elif len(query) == 3:
            print("corrections for", query[0], ":")
            print(result[0])
            print("corrections for", query[2], ":")
            print(result[1], "\n")
            
            # Handling AND case
            if query[1] == 'AND':
                for term1 in result[0]:
                    for term2 in result[1]:
                        print("results for", term1, " AND ", term2, ":")
                        handling_AND(term1, term2, trie)
                print("done!")
                return
            
            # Handling OR case
            elif query[1] == 'OR':
                for term1 in result[0]:
                    for term2 in result[1]:
                        print("results for", term1, " OR ", term2, ":")
                        handling_OR(term1, term2, trie)
                print("done!")
                return
            
            # Handling proximity case
            elif 'NEAR/' in query[1]:
                for term1 in result[0]:
                    for term2 in result[1]:
                        print("results for", term1, query[1], term2, ":")
                        handling_proximity(term1, term2, int(query[1][5:]), trie)
                print("done!")
                return
    
    # Correction case
    else:
        # Finding the correct way of each term
        result = []
        for word in query:
            if "*" in word:
                result.append([make_original(term) for term in wildcard(word.lower(), trie)])
            else:
                result.append(spell_correction(word.lower(), tokenized_list_stop_words))
                
        print("here are lists for each word:")
        for i in range(len(result)):
            print(query[i], ":")
            print(result[i])
        print("\nand here are the results:")
        
        # Finding the cartesian product of all correct ways
        cartesian_product = list(product(*result))

        # Print the result
        for item in cartesian_product:
            print(" ".join(item))

This function is used to convert a matrix to a set to get all the unique terms.

In [ ]:
def find_unique(matrix):
    # Converting a matrix to a set
    l = []
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            l.append(matrix[i][j])
    return set(l)

# Main:

1. First of all we get text_files and documents using get_text_files function based on available documents.
2. Use the tokenize function to get the tokenized list.
3. Get all unique terms form documents using make_set function.
4. Construct the Trie.
5. Get the query from user.

In [ ]:
# Getting documents and file names
text_files, documents = get_text_files()
# tokenize every document
tokenized_list = tokenize(documents)
# Unique terms
tokenized_list_stop_words =  list(find_unique(tokenized_list))
# build the Trie
trie = construct_trie(tokenized_list)

# Getting the query and start searching
information_retreival(list(input("Enter what you are looking for ('Enter' for stoping the search)").split()), trie)